# Import danych z użyciem Kaggle API

Aby móc pobrać dataset automatycznie, należy wprowadzić swoją nazwę użytkownika oraz klucz API z nim związany.

INFO: https://www.kaggle.com/docs/api#authentication

In [1]:
import os

# From kaggle.json file, paste the values here.
os.environ['KAGGLE_USERNAME'] = "username"  # username
os.environ['KAGGLE_KEY']      = "api_key"   # key

In [2]:
!pip install kaggle
!kaggle datasets download -d ahemateja19bec1025/traffic-sign-dataset-classification

Dataset URL: https://www.kaggle.com/datasets/ahemateja19bec1025/traffic-sign-dataset-classification
License(s): CC0-1.0
 65% 123M/190M [00:00<00:00, 1.28GB/s]
100% 190M/190M [00:00<00:00, 788MB/s] 


Ekstrakcja danych

In [3]:
import zipfile

filename = "traffic-sign-dataset-classification.zip"
with zipfile.ZipFile(filename, "r") as archive:
    archive.extractall("dataset")

# Wczytywanie zbioru danych

In [48]:
import keras

BATCH_SIZE = 32
IMG_SIZE   = (256, 256)

train_set = keras.utils.image_dataset_from_directory(
    directory='dataset/traffic_Data/DATA',
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

test_set = keras.utils.image_dataset_from_directory(
    directory='dataset/traffic_Data/TEST',
    labels=None,
    batch_size=None,    # Ważne! W dalszym etapie będzie spinanie batchem.
    image_size=IMG_SIZE
)

Found 4170 files belonging to 58 classes.
Found 1994 files.


Łączenie danych testowych z etykietami (są tam tylko zdjęcia zamiast segregacji między folderami jako klasy)

In [49]:
import pandas as pd
import tensorflow as tf

df = pd.read_csv("dataset/labels.csv") \
       .sort_values("Name") \
       .reset_index(drop=True)

num_classes = train_set.element_spec[1].shape[-1]
labels_int  = df["ClassId"].astype(int).tolist()
labels_oh   = tf.one_hot(labels_int, depth=num_classes)

label_ds = tf.data.Dataset.from_tensor_slices(labels_oh)
test_set = tf.data.Dataset.zip((test_set, label_ds)).batch(BATCH_SIZE)

In [50]:
num_classes = train_set.element_spec[1].shape[-1]

model = keras.Sequential([
    keras.layers.InputLayer(input_shape=IMG_SIZE+(3,)),
    keras.layers.Rescaling(1./255),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax'),
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [51]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_8 (Rescaling)         │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │    31,490,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 58)             │         7,482 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,517,050 (120.23 MB)

 Trainable params: 31,517,050 (120.23 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
model.compile(
    optimizer = 'adam',
    loss      = 'categorical_crossentropy',
    metrics   = ['accuracy']
)

In [54]:
EPOCHS = 2

history = model.fit(
    train_set,
    epochs          = EPOCHS,
    validation_data = test_set
)

Epoch 1/2
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 1.0000 - loss: 6.3293e-04 - val_accuracy: 0.0000e+00 - val_loss: 23.3164
Epoch 2/2
131/131 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - accuracy: 1.0000 - loss: 2.9970e-04 - val_accuracy: 0.0172 - val_loss: 21.9078
